## Docker basics
You just learned how containers are revolutionizing the way software organizations build, test, and deploy software. You've also learned about a container runtime called Docker, which provides an easy interface to package software in the containers and deploy them across one or more different platforms.

Let's start by running docker --version to confirm that both the client and server are there and working.

In [ ]:
!docker --version

Docker containers are built using images. Let's run the command docker pull nginx:latest to pull down the latest nginx trusted image from Docker Hub.

In [ ]:
!docker pull nginx:latest

Now run docker images to verify that the image is now on your local machine's Docker cache. If we start it then it won't have to pull it down from Docker Hub first.

In [ ]:
!docker images

Now let's try docker run -d -p 8080:80 --name nginx nginx:latest to instantiate the nginx image as a background daemon with port 8080 on the host forwarding through to port 80 within the container

In [ ]:
!docker run -d -p 8080:80 --name nginx nginx:latest

Run docker ps to see that our nginx container is running.

In [ ]:
!docker ps

Try curl http://localhost:8080 to use the nginx container and verify it is working with its default index.html.

In [ ]:
!curl http://localhost:8080

Running docker logs nginx shows us the logs produced by nginx and the container. You should see some events that correspond to our curl requests.

In [ ]:
!docker logs nginx

Use docker exec -it nginx /bin/bash to start an interactive shell into the container's filesystem and constraints

In [ ]:
!docker exec -it nginx /bin/bash

From within the container, run cd /usr/share/nginx/html and cat index.html to see the content the nginx is serving which is part of the container.

In [ ]:
!cd /usr/share/nginx/html
!cat index.html

Type exit to exit our shell within the container.

In [ ]:
!exit

Now run docker stop nginx to stop the container.

In [ ]:
!docker stop nginx

Try docker ps -a command and you should see that our container is still there but stopped. At this point it could be restarted with a docker start nginx if we wanted.

In [ ]:
!docker ps -a

To remove the container once and for all, use the docker rm nginx command.

In [ ]:
!docker rm nginx

Now you can use docker rmi nginx:latest to remove the nginx image from our machine's local cache

In [ ]:
!docker rmi nginx:latest

### Building a container image
In this step, you use a Dockerfile to build a container image onto the instance. This sample uses an image that includes the Nginx webserver to serve a simple html page. Let's start by creating a working directory for our container image

In [ ]:
!mkdir container-image
!cd container-image/

Run the below command to update the Dockerfile content

In [ ]:
!cat <<EOF > Dockerfile
FROM nginx:latest
COPY index.html /usr/share/nginx/html
EOF

In [ ]:
!echo "We've added our own custom content into the container" >> index.html

Use docker build -t nginx:1.0 . to build the nginx container image from our Dockerfile

In [ ]:
!docker build -t nginx:1.0 .

You can now use docker history nginx:1.0 to see all the steps and base containers that our nginx:1.0 is built on. Note that our change amounts to one new tiny layer on top.

In [ ]:
!docker history nginx:1.0

Type docker run -p 8080:80 --name nginx nginx:1.0 to run our new container. Note that we didn't specify the -d to make it a daemon which means it holds control of our terminal and outputs the containers logs to there which can be handy in debugging.

In [ ]:
!docker run -p 8080:80 --name nginx nginx:1.0

Open another Terminal tab (Window -> New Terminal)<br>
Run curl http://localhost:8080 in the other tab a few times and see our new content.

In [ ]:
!curl http://localhost:8080

Go back to the first tab and see the log lines sent right out to STDOUT.<br>
At this point we could push it to Docker Hub or a private registry like Amazon ECR  for others to pull and run. You will learn more about that shortly.<br>
Type Ctrl-C to exit the log output. Note that the container has been stopped but is still there by running a docker ps -a.

In [ ]:
!docker ps -a

Use sudo docker inspect nginx to see lots of info about our stopped container.

In [ ]:
!sudo docker inspect nginx

For our last magic trick, we're going to try mounting some files from the host into the container rather than embedding them in the image.

In [ ]:
!docker run -d -p 8080:80 -v /home/ec2-user/environment/container-image/index.html:/usr/share/nginx/html/index.html\:ro --name nginx nginx:latest

Now try a curl http://localhost:8080. Note that even though this is the upstream nginx image from Docker Hub our content is there.

In [ ]:
!curl http://localhost:8080

Finally, run docker stop nginx and docker rm nginx to stop and remove our last container.

In [ ]:
!docker stop nginx && docker rm nginx